### JSON Output Parser

In [1]:
from langchain_core.output_parsers.json import JsonOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models import ChatOpenAI

In [2]:
import getpass
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API Key")

In [ ]:
llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model='gpt-3.5-turbo-1106')

In [4]:
class Answer_with_Source(BaseModel):
    answer: str = Field(description="exact answer to the question")
    source: str = Field(description="source wesbite of the answer")

In [5]:
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Answer_with_Source)
format_response_parser = parser.get_format_instructions()

In [97]:
prompt_template = '''
    Answer the query. \n
    {format_instructions} \n
    Question: {question}
    '''

In [98]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_response_parser},
)

chain = prompt | llm_openai | parser

In [99]:
chain.invoke({"question": "who won the first IPL?"})

{'answer': 'Rajasthan Royals',
 'source': 'https://www.iplt20.com/stats/2021/most-runs'}

In [100]:
chain.invoke({"question": "What is the capital of Japan"})

{'answer': 'Tokyo', 'source': 'https://en.wikipedia.org/wiki/Tokyo'}

Supports Streaming

In [101]:
for c in chain.stream({"question": "What is the GDP of India?"}):
    print(c)

{}
{'answer': ''}
{'answer': 'The'}
{'answer': 'The GDP'}
{'answer': 'The GDP of'}
{'answer': 'The GDP of India'}
{'answer': 'The GDP of India is'}
{'answer': 'The GDP of India is $'}
{'answer': 'The GDP of India is $2'}
{'answer': 'The GDP of India is $2.'}
{'answer': 'The GDP of India is $2.87'}
{'answer': 'The GDP of India is $2.87 trillion'}
{'answer': 'The GDP of India is $2.87 trillion.'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': ''}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https://'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https://www'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https://www.world'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https://www.worldbank'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https://www.worldbank.org'}
{'answer': 'The GDP of India is $2.87 trillion.', 'source': 'https:

Let's try without pydantic

In [102]:
parser_w_pydantic = JsonOutputParser()

In [103]:
prompt_w_pydantic = PromptTemplate(
    template=prompt_template,
    input_variables=["question"],
    partial_variables={"format_instructions": parser_w_pydantic},
)

chain_w_pydantic = prompt | llm_openai | parser_w_pydantic

In [104]:
chain.invoke({"question": "why amazon is better than flipkart"})

{'answer': 'The answer to this question is subjective and depends on individual preferences and experiences. Some people may prefer Amazon for its wider product selection and faster delivery, while others may prefer Flipkart for its user interface and customer service. It is important to consider personal needs and priorities when choosing between the two.',
 'source': 'N/A - This is a subjective question and does not have a specific source for the answer'}

### Structured Output Parser

In [105]:
from langchain.output_parsers.structured import ResponseSchema, StructuredOutputParser

In [106]:
resp_schema_for_answer = ResponseSchema(name="answer", description="answer to the user's question")
resp_schema_for_source = ResponseSchema(name="source",description="source used to answer")
resp_schema_for_source_link = ResponseSchema(name="source_with_link",description="source used to answer the user's question, should be a website.")

In [107]:
response_schemas = [
    resp_schema_for_answer,
    resp_schema_for_source,
    resp_schema_for_source_link,
]
std_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [108]:
std_format_response_parser = std_output_parser.get_format_instructions()

In [110]:
std_output_resp_prompt_template = '''
    Answer the query. \n
    {format_instructions} \n
    Question: {question}
    '''

In [113]:
std_output_resp_prompt = PromptTemplate(
    template=std_output_resp_prompt_template,
    input_variables=["question"],
    partial_variables={"format_instructions": std_format_response_parser},
)

std_output_chain = std_output_resp_prompt | llm_openai | std_output_parser

In [115]:
std_output_chain.invoke({"question": "Who is the captain of Indian Cricket team?"})

{'answer': 'Virat Kohli is the captain of the Indian Cricket team.',
 'source': 'Various sports news outlets',
 'source_with_link': 'https://www.espncricinfo.com/team/india-6'}

In [116]:
std_output_chain.invoke({"question": "What is the fastest animal in the world?"})

{'answer': 'The fastest animal in the world is the peregrine falcon, which can reach speeds of up to 240 miles per hour when diving.',
 'source': 'National Geographic',
 'source_with_link': 'https://www.nationalgeographic.com/animals/birds/facts/peregrine-falcon'}